In [1]:
from flask import Flask, request, jsonify
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize the Flask application
app = Flask(__name__)

# Set the API key from the environment variable
api_key = os.getenv('API_KEY')

# Set the API endpoint for the generative language model
endpoint = "https://generativelanguage.googleapis.com/v1beta"

@app.route('/')
def index():
    return 'Hello, World!'

@app.route('/chat', methods=['POST'])
def chat_with_coach():
    try:
        # Ensure the request has JSON data
        if not request.is_json:
            return jsonify({"error": "Invalid input, expected JSON"}), 400

        # Get data from the incoming request
        data = request.json

        # Retrieve scenario and user message
        scenario = data.get("scenario")
        user_message = data.get("message")

        # Validate input
        if not scenario or not user_message:
            return jsonify({"error": "Missing 'scenario' or 'message'"}), 400

        # Construct the prompt for the AI model
        prompt = f"You are an AI coach helping users with English in engineering contexts. Scenario: {scenario}. User said: '{user_message}'. Provide constructive feedback, correct any mistakes, and give suggestions to improve."

        # Prepare the request body for the Generative Language API
        request_body = {
            "contents": [
                {
                    "parts": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ]
        }

        # Make the API request to Generative Language API
        response = requests.post(
            f"{endpoint}/models/gemini-pro:generateContent?key={api_key}",
            json=request_body
        )

        # Check for a successful response from the AI API
        if response.status_code == 200:
            result = response.json()
            # Extract the feedback from the response
            feedback = result.get("contents", [{}])[0].get("parts", [{}])[0].get("text", "")
            return jsonify({"feedback": feedback})
        else:
            # Handle unsuccessful API response
            return jsonify({"error": "Failed to retrieve response from AI"}), response.status_code

    except Exception as e:
        # Catch and handle unexpected errors
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    # Set the port to 5001 explicitly
    port = 5001
    print(f"Running on http://127.0.0.1:{port}")
    app.run(debug=True, port=port)


Running on http://127.0.0.1:5001
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ash93/projects/lingineer/server/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/ash93/projects/lingineer/server/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/ash93/projects/lingineer/server/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/ash93/projects/lingineer/server/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/ash93/pr

SystemExit: 1

/home/ash93/projects/lingineer/server/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
